# Helper REST functions for DataScope Select

In [1]:
import requests, json

In [10]:
rthBaseURL = 'https://selectapi.datascope.refinitiv.com/RestApi/v1'

hdrs = {
    "Prefer": "respond-async",
    "Content-Type": "application/json" 
}

In [11]:
def DSS_login(rthUserName, rthPassword):
    global hdrs
    pData = {
        "Credentials": {
            "Username": rthUserName,
            "Password": rthPassword
        }
    }

    dResp = requests.post(f'{rthBaseURL}/Authentication/RequestToken', data=json.dumps(pData), headers=hdrs)

    if (dResp.status_code == 200):
        aToken = dResp.json()['value']
        # keep a copy of the oAuth token for other calls
        hdrs["Authorization"] = "Token " + aToken
        return aToken
    else:
        raise ValueError(dResp.json()['error']['message'])


In [14]:
def DSS_getAsync(uri):
    dResp = requests.get(f'{rthBaseURL}{uri}', headers=hdrs)
    # is it a 202 (in progress)
    while dResp.status_code == 202:
        print("waiting for response...")
        # check if the response completed yet
        nURL = dResp.headers['Location']
        dResp = requests.get(nURL, headers=hdrs)

    if (dResp.status_code == 200):
        return dResp.json()
    else:
        raise ValueError(dResp.json())


In [ ]:
def DSS_postAsync(uri, pData):
    dResp = requests.post(f'{rthBaseURL}{uri}', data=json.dumps(pData), headers=hdrs)
    # is it a 202 (in progress)
    while dResp.status_code == 202:
        print("waiting for response...")
        # check if the response completed yet
        nURL = dResp.headers['Location']
        dResp = requests.get(nURL, headers=hdrs)

    if (dResp.status_code == 200):
        return dResp.json()
    else:
        raise ValueError(dResp.json())
